In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
model_name_or_path = '/data1/models/Llama-2-7b-chat-hf'
mask_model_name_or_path = '/data1/models/t5-base'
from mgtbench import AutoDetector, AutoExperiment
from mgtbench.loading.dataloader import load
detectGPT = AutoDetector.from_detector_name('detectGPT', 
                                            model_name_or_path=model_name_or_path,
                                            mask_model_name_or_path= mask_model_name_or_path)
experiment = AutoExperiment.from_experiment_name('perturb',detector=[detectGPT])
data = load('Essay', 'ChatGLM')
data['train']['text']   =data['train']['text'] [:20]
data['train']['label']=data['train']['label'][:20]
data['test']['text']=data['test']['text'][:20]
data['test']['label']=data['test']['label'][:20]
experiment.load_data(data)
experiment.launch(n_perturbations=2)

In [3]:
model_name_or_path = '/data1/models/Llama-2-7b-chat-hf'
from mgtbench import AutoDetector, AutoExperiment
from mgtbench.loading.dataloader import load
metric1 = AutoDetector.from_detector_name('ll', 
                                            model_name_or_path=model_name_or_path)
metric2 = AutoDetector.from_detector_name('rank', 
                                            model_name_or_path=model_name_or_path)
experiment = AutoExperiment.from_experiment_name('threshold',detector=[metric1, metric2])
data = load('Essay', 'ChatGLM')
data['train']['text']   =data['train']['text'] [:20]
data['train']['label']=data['train']['label'][:20]
data['test']['text']=data['test']['text'][:20]
data['test']['label']=data['test']['label'][:20]
experiment.load_data(data)
experiment.launch()

[2024-07-18 17:05:50,349] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

parsing data: 100%|██████████| 997/997 [00:00<00:00, 10399.73it/s]


Calculate result for each data point
Running prediction of detector ll
Predict training data


...

100%|██████████| 20/20 [00:02<00:00,  9.51it/s]


Predict testing data


100%|██████████| 20/20 [00:02<00:00,  9.53it/s]


Run classification for results


[DetectOutput(name='', train=Metric(acc=0.9, precision=0.9, recall=0.9, f1=0.9, auc=0.99), test=Metric(acc=0.9, precision=0.8333333333333334, recall=1.0, f1=0.9090909090909091, auc=0.9500000000000001)),
 DetectOutput(name='', train=Metric(acc=0.75, precision=0.7272727272727273, recall=0.8, f1=0.7619047619047619, auc=0.87), test=Metric(acc=0.6, precision=0.625, recall=0.5, f1=0.5555555555555556, auc=0.73))]

In [4]:
from mgtbench import AutoDetector, AutoExperiment
d = AutoDetector.from_detector_name('demasq', state_dict_path='model_weight')
e = AutoExperiment.from_experiment_name('demasq',detector=[d])
from mgtbench.loading.dataloader import load

data = load('Essay', 'ChatGLM')
data['train']['text']   =data['train']['text'] [:200]
data['train']['label']=data['train']['label'][:200]
data['test']['text']=data['test']['text'][:20]
data['test']['label']=data['test']['label'][:20]
len(data['train']['text'][0].split())
e.load_data(data)
e.launch(epoch=2,need_finetune=True)


parsing data: 100%|██████████| 997/997 [00:00<00:00, 10431.04it/s]


Calculate result for each data point
Running prediction of detector demasq


100%|██████████| 180/180 [00:08<00:00, 20.94it/s]


Epoch [1/2], Loss: 117.37925746122168
Epoch [1/2], Val: f1 1.0


100%|██████████| 180/180 [00:08<00:00, 21.69it/s]


Epoch [2/2], Loss: 117.34796204075553
Epoch [2/2], Val: f1 0.9166666666666666


[DetectOutput(name='', train=Metric(acc=0.99, precision=0.9803921568627451, recall=1.0, f1=0.9900990099009901, auc=1.0), test=Metric(acc=0.95, precision=1.0, recall=0.9, f1=0.9473684210526315, auc=0.96))]

In [11]:
from mgtbench import AutoDetector, AutoExperiment
detector = AutoDetector.from_detector_name('supervised', model_name_or_path='/data1/models/distilbert-base-uncased')
experiment = AutoExperiment.from_experiment_name('supervised',detector=[detector])


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /data1/models/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
detector.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [13]:
detector.detect('hello')

100%|██████████| 1/1 [00:00<00:00, 206.54it/s]


[[0.50146484375]]

In [10]:
from dataclasses import dataclass, fields, asdict
@dataclass
class A:
    span_length:int = 2

    def update(self, kargs):
        for field in fields(self):
            if field.name in kargs:
                setattr(self, field.name, kargs[field.name])
aa = A()
aa.update({'span_length':1})

In [11]:
aa

A(span_length=1)

[2024-07-18 16:53:16,625] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

parsing data: 100%|██████████| 997/997 [00:00<00:00, 10256.56it/s]


Calculate result for each data point
Running prediction of detector detectGPT
Predict training data
Running perturb on the given texts


100%|██████████| 2/2 [00:05<00:00,  2.59s/it]


...

100%|██████████| 2/2 [00:07<00:00,  3.98s/it]


Perturb finished.


100%|██████████| 40/40 [00:04<00:00,  9.75it/s]

Run classification for results


[DetectOutput(name='', train=Metric(acc=0.9, precision=0.9, recall=0.9, f1=0.9, auc=0.9199999999999999), test=Metric(acc=0.85, precision=0.8181818181818182, recall=0.9, f1=0.8571428571428571, auc=0.92))]

In [14]:
model_name_or_path = '/data1/models/distilbert-base-uncased'
from mgtbench import AutoDetector, AutoExperiment
from mgtbench.loading.dataloader import load

detector = AutoDetector.from_detector_name('supervised', model_name_or_path=model_name_or_path)
experiment = AutoExperiment.from_experiment_name('supervised',detector=[detector])
data = load('Essay', 'ChatGLM')
data['train']['text']   =data['train']['text'] [:20]
data['train']['label']=data['train']['label'][:20]
data['test']['text']=data['test']['text'][:20]
data['test']['label']=data['test']['label'][:20]
experiment.load_data(data)
experiment.launch(n_perturbations=2)


parsing data: 100%|██████████| 997/997 [00:00<00:00, 10325.06it/s]


parsing data: 100%|██████████| 997/997 [00:00<00:00, 10423.87it/s]


In [17]:
experiment.load_data(data)
experiment.loaded

True

In [13]:
experiment.perturb_config

PerturbConfig(span_length=2, buffer_size=1, mask_top_p=1, pct_words_masked=0.3)

In [18]:
experiment.launch()

Calculate result for each data point
Running prediction of detector LRR
Predict training data


  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 13.81it/s]


Predict testing data


100%|██████████| 2/2 [00:00<00:00,  8.97it/s]

Run classification for results



/home/lyl/miniconda3/envs/llm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[DetectOutput(name='', train=Metric(acc=1.0, precision=1.0, recall=1.0, f1=1.0, auc=1.0), test=Metric(acc=0.5, precision=0.0, recall=0.0, f1=0.0, auc=1.0))]

In [23]:
import torch
import numpy as np
a = torch.Tensor([-2.1801, -2.1619])
a = a.numpy()
np.mean(a)

-2.171

In [24]:
a = np.array([1,2])
b = np.array([2, 3])
a/b

array([0.5       , 0.66666667])

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class DEMASQ(nn.Module):
    def __init__(self):
        super(DEMASQ, self).__init__()
        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x



In [19]:
from scipy.special import jn_zeros

def compute_zeros(d, c_n):
    m = d
    n = c_n
    drumhead_fundamental_f = jn_zeros(0,1)
    drumhead_f = jn_zeros(m, n+1)
    return drumhead_f[c_n]//drumhead_fundamental_f

def source_frequency(emb):
    min_emb = torch.min(emb)
    max_emb = torch.max(emb)
    if min_emb<0:
        emb = emb + min_emb
    else:
        emb = emb - max_emb
    unique_values = torch.unique(emb)
    return compute_zeros(0, len(unique_values))

def enery(emb, y):
    c = torch.var(emb).cpu()
    v_r = (1-y)*abs(c)
    v_s = 0.8
    E_f0 = source_frequency(emb)
    E_f = (c+v_r)/(c-v_s)*E_f0
    return E_f


In [3]:
from mgtbench.loading.dataloader import load
from  sentence_transformers import SentenceTransformer
data = load('Essay', 'ChatGLM')
model = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')
train_embs = model.encode(data['train']['text'], convert_to_tensor=True)
test_embs = model.encode(data['test']['text'], convert_to_tensor=True)
train = list(zip(train_embs, data['train']['label']))
test = list(zip(test_embs, data['test']['label']))

parsing data: 100%|██████████| 997/997 [00:00<00:00, 10402.86it/s]


In [4]:
num_epochs = 12
from torch.utils.data import DataLoader
# Instantiate the model
demasq = DEMASQ()
demasq.cuda()
optimizer = optim.Adam(demasq.parameters(), lr=0.0001)
criterion = nn.BCELoss()


In [5]:
output = demasq(torch.unsqueeze(train[0][0],dim=0).cuda())
m = nn.Sigmoid()
criterion(m(output), torch.Tensor([[train[0][1]]]).cuda())

tensor(0.7057, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [16]:
e.cuda()

tensor([0.], device='cuda:0', dtype=torch.float64)